In [41]:
%reset

In [2]:
import aegis
import numpy as np
import healpy as hp
import torch
import pickle as pk
from astropy import units as u
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
import sys
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
import pickle
from scipy.stats import norm
from scipy.integrate import quad, simpson
from joblib import Parallel, delayed

%matplotlib inline

In [3]:
grains=1000
num_simulations = 1000
num_workers = -1

In [4]:
parameter_range = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [2000, 100000] #MeV
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
max_radius = 8.5 + 20*2 #kpc
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [5]:
my_cosmology = 'Planck18'
z_range = [0, 14]
luminosity_range = 10.0**np.array([37, 50]) # Minimum value set by considering Andromeda distance using Fermi as benchmark and receiving 0.1 photon at detector side
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, cosmology = my_cosmology, z_range = z_range, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen, my_AEGIS.Emin_gen, my_AEGIS.Emax_gen = angular_cut_gen, lat_cut_gen, energy_range_gen[0], energy_range_gen[1]

!!!WARNING!!! Some high energy photons that could be redshifted into the final energy range may not be generated. It is recommended to increase the maximum generating energy


In [6]:
Gamma_SFG = 2.2
gamma_energy_bounds = np.array([0.1, 1000])  # in GeV
E_photon_GeV_SFG = ((-Gamma_SFG + 1) / (-Gamma_SFG + 2) *
                (gamma_energy_bounds[1]**(-Gamma_SFG + 2) - gamma_energy_bounds[0]**(-Gamma_SFG + 2)) /
                (gamma_energy_bounds[1]**(-Gamma_SFG + 1) - gamma_energy_bounds[0]**(-Gamma_SFG + 1)))
E_photon_SFG = E_photon_GeV_SFG * 0.00160218  # erg

Gamma_mAGN = 2.25
gamma_energy_bounds = np.array([0.1, 1000])  # in GeV
E_photon_GeV_mAGN = ((-Gamma_mAGN + 1) / (-Gamma_mAGN + 2) *
                (gamma_energy_bounds[1]**(-Gamma_mAGN + 2) - gamma_energy_bounds[0]**(-Gamma_mAGN + 2)) /
                (gamma_energy_bounds[1]**(-Gamma_mAGN + 1) - gamma_energy_bounds[0]**(-Gamma_mAGN + 1)))
E_photon_mAGN = E_photon_GeV_mAGN * 0.00160218  # erg

res = int(1e4)
log_LIRs = np.linspace(-5, 25, res)
log_L5Gs = np.linspace(20, 55, res)

In [7]:
def ZL_SFG1(z, l, params):


    log_PhiStar = params[0]
    Phi_star = 10**log_PhiStar

    l_erg = l * E_photon_SFG # erg/s
    LFs = np.zeros_like(l)

    def Phi_IR(log_LIR): #log_LIR = log_10(L_IR / solar_luminosity) # unitless

        # from Table 8 in Gruppioni et al.
        # Phi_star = 10**(-2.08) # Mpc^{-3} dex^{-1}
        Lstar = 10**(9.46) # Solar luminosity
        alpha = 1.00
        sigma = 0.50

        LIR = 10**log_LIR # solar luminosity

        Phi_IR = Phi_star * (LIR / Lstar)**(1 - alpha) * np.exp(-1 / (2 * sigma**2) * (np.log10(1 + LIR / Lstar))**2) # from Gruppioni paper eqn (3)  	

        return Phi_IR

    def PDF_log_Lgamma_given_log_LIR(log_LIR, log_Lgamma): #log_LIR = log_10(L_IR / solar_luminosity) # unitless
        LIR_solar_luminosity = 10**log_LIR # Solar luminosity
        L_IR_erg_second = LIR_solar_luminosity * 3.826e33 # erg/s

        a = 1.09
        g = 40.8
        sigma_SF = 0.202 

        mean = g + a * np.log10(L_IR_erg_second / 1e45)
        std = sigma_SF

        return norm.pdf(log_Lgamma, loc=mean, scale=std)

    def integrand(PhiIR_of_logLIRs, log_LIRs, log_Lgamma):
        return PhiIR_of_logLIRs * PDF_log_Lgamma_given_log_LIR(log_LIRs, log_Lgamma)

    PhiIR_of_logLIRs = Phi_IR(log_LIRs)

    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(PhiIR_of_logLIRs, log_LIRs, np.log10(l_erg[i,j])), x=log_LIRs)
    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9


def spec_SFG1(energy, params):
    Gamma = 2.2
    return energy**(-Gamma)

In [8]:
def ZL_SFG2(z, l, params):


    log_PhiStar = params[1]
    Phi_star = 10**log_PhiStar

    l_erg = l * E_photon_SFG # erg/s
    LFs = np.zeros_like(l)

    def Phi_IR(log_LIR): #log_LIR = log_10(L_IR / solar_luminosity) # unitless

        # from Table 8 in Gruppioni et al.
        # Phi_star = 10**(−4.74) # Mpc^{-3} dex^{-1}
        Lstar = 10**(11.02) # Solar luminosity
        alpha = 1.00
        sigma = 0.35

        LIR = 10**log_LIR # solar luminosity

        Phi_IR = Phi_star * (LIR / Lstar)**(1 - alpha) * np.exp(-1 / (2 * sigma**2) * (np.log10(1 + LIR / Lstar))**2) # from Gruppioni paper eqn (3)  	

        return Phi_IR

    def PDF_log_Lgamma_given_log_LIR(log_LIR, log_Lgamma): #log_LIR = log_10(L_IR / solar_luminosity) # unitless
        LIR_solar_luminosity = 10**log_LIR # Solar luminosity
        L_IR_erg_second = LIR_solar_luminosity * 3.826e33 # erg/s

        a = 1.09
        g = 40.8
        sigma_SF = 0.202 

        mean = g + a * np.log10(L_IR_erg_second / 1e45)
        std = sigma_SF

        return norm.pdf(log_Lgamma, loc=mean, scale=std)

    def integrand(PhiIR_of_logLIRs, log_LIRs, log_Lgamma):
        return PhiIR_of_logLIRs * PDF_log_Lgamma_given_log_LIR(log_LIRs, log_Lgamma)

    PhiIR_of_logLIRs = Phi_IR(log_LIRs)

    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(PhiIR_of_logLIRs, log_LIRs, np.log10(l_erg[i,j])), x=log_LIRs)
    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9


def spec_SFG2(energy, params):
    Gamma = 2.2
    return energy**(-Gamma)

In [9]:
def ZL_SFG3(z, l, params):


    log_PhiStar = params[2]
    Phi_star = 10**log_PhiStar

    l_erg = l * E_photon_SFG # erg/s
    LFs = np.zeros_like(l)

    def Phi_IR(log_LIR): #log_LIR = log_10(L_IR / solar_luminosity) # unitless

        # from Table 8 in Gruppioni et al.
        # Phi_star = 10**(−3.25) # Mpc^{-3} dex^{-1}
        Lstar = 10**(10.57) # Solar luminosity
        alpha = 1.2
        sigma = 0.4

        LIR = 10**log_LIR # solar luminosity

        Phi_IR = Phi_star * (LIR / Lstar)**(1 - alpha) * np.exp(-1 / (2 * sigma**2) * (np.log10(1 + LIR / Lstar))**2) # from Gruppioni paper eqn (3)  	

        return Phi_IR

    def PDF_log_Lgamma_given_log_LIR(log_LIR, log_Lgamma): #log_LIR = log_10(L_IR / solar_luminosity) # unitless
        LIR_solar_luminosity = 10**log_LIR # Solar luminosity
        L_IR_erg_second = LIR_solar_luminosity * 3.826e33 # erg/s

        a = 1.09
        g = 40.8
        sigma_SF = 0.202 

        mean = g + a * np.log10(L_IR_erg_second / 1e45)
        std = sigma_SF

        return norm.pdf(log_Lgamma, loc=mean, scale=std)

    def integrand(PhiIR_of_logLIRs, log_LIRs, log_Lgamma):
        return PhiIR_of_logLIRs * PDF_log_Lgamma_given_log_LIR(log_LIRs, log_Lgamma)

    PhiIR_of_logLIRs = Phi_IR(log_LIRs)

    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(PhiIR_of_logLIRs, log_LIRs, np.log10(l_erg[i,j])), x=log_LIRs)
    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9


def spec_SFG3(energy, params):
    Gamma = 2.2
    return energy**(-Gamma)

In [10]:
def ZL_mAGN(z, l, params):

    log_phi1 = params[3]
    phi1 = 10**log_phi1

    l_erg = l * E_photon_mAGN # erg/s
    LFs = np.zeros_like(l)

    def Phi_5G(log_L5G, z): #log_L5G = log_10(L_5GHz / (erg/s)) # unitless
        #Output is in Mpc^{-3}

        L_5G = 10**log_L5G # erg/s
        radio_bandwidth = 4.87e9 # measured in Hz # width of radio band centered around blueshifted frequency of 5GHz 
        diff_L5G = L_5G / radio_bandwidth * 1e-7 # measured in W/Hz # Converted erg to Joule # luminosity per unit frequency

        # Values taken from Table 4 of Yuan 2018 paper. Second row.
        p1 = 2.085
        p2 = -4.602
        z_c = 0.893
        k1 = 1.744
        e1 = ( (1+z_c)**p1 + (1+z_c)**p2 ) / ( ((1+z_c)/(1+z))**p1 + ((1+z_c)/(1+z))**p2 )
        e2 = (1+z)**k1
        # phi1 = 10**(-3.749) # Mpc^{-3}
        L_star = 10**21.592 # W/Hz
        beta = 0.139
        gamma = 0.878

        # From Yuan 2018 paper equation 21
        # Note that this is dN/dV dlog(diff_5G). But this is also equal to dN/dV dlog(L_5G) because the radio bandwidth is fixed.
        Phi_5G = e1 * phi1 * ( (diff_L5G / (e2 * L_star))**beta + (diff_L5G / (e2 * L_star))**gamma )**-1

        return Phi_5G
    

    def PDF_log_Lgamma_given_log_L5G(log_L5G, log_Lgamma): #log_L5G = log_10(L_5GHz / (erg/s)) # unitless
        L_5GHz = 10**log_L5G # erg/s

        b = 0.78
        d = 40.78
        sigma_mAGN = 0.880

        mean = d + b * np.log10(L_5GHz / 1e40)
        std = sigma_mAGN

        return norm.pdf(log_Lgamma, loc=mean, scale=std)
    

    def integrand(log_L5G, z, log_Lgamma):
        return Phi_5G(log_L5G, z) * PDF_log_Lgamma_given_log_L5G(log_L5G, log_Lgamma)
    


    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(log_L5Gs, z[i,j], np.log10(l_erg[i,j])), x=log_L5Gs)


    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9



def spec_mAGN(energy, params):
    Gamma = 2.25
    return energy**(-Gamma)

In [11]:
als_SFG1 = [ZL_SFG1, spec_SFG1]
als_SFG2 = [ZL_SFG2, spec_SFG2]
als_SFG3 = [ZL_SFG3, spec_SFG3]
als_mAGN = [ZL_mAGN, spec_mAGN]
my_AEGIS.abun_lum_spec = [als_SFG1, als_SFG2, als_SFG3, als_mAGN]
my_AEGIS.source_class_list = ['extragalactic_isotropic_faint_single_spectrum', 'extragalactic_isotropic_faint_single_spectrum', 'extragalactic_isotropic_faint_single_spectrum', 'extragalactic_isotropic_faint_single_spectrum']

In [52]:
# def compute_moments(energies):
#     """Compute the mean and variance of the energies."""
#     mean_E = np.mean(energies)
#     var_E = np.var(energies)
#     return mean_E, var_E

# def compute_quantiles(energies, quantiles=[10, 25, 50, 75, 90]):
#     """
#     Compute the specified quantiles (in percent).
#     For example, the 25th quantile is the energy such that 25% of the data lies below it.
#     Returns a dictionary mapping percentiles to values.
#     """
#     q_values = np.percentile(energies, quantiles)
#     return dict(zip(quantiles, q_values))


# def compute_energy_only_histogram(energies, num_bins, energy_range=energy_range):
#     """
#     Compute a 1D histogram of energies using logarithmic binning.
#     Returns the histogram counts and the bin edges.
#     """
#     bins = np.geomspace(energy_range[0], energy_range[1], num_bins + 1)
#     hist, _ = np.histogram(energies, bins=bins)
#     return hist

# def effective_spectral_index(energies, E_lower, min_count=6):
#     """
#     Compute effective spectral index via MLE, or return -1 if not enough photons.
#     """
#     energies = np.array(energies)
#     n = len(energies)
#     if n < min_count:
#         return -100.0  # Sentinel value indicating unreliable or undefined
#     sum_logs = np.sum(np.log(energies / E_lower))
#     gamma_hat = 1 + n / sum_logs
#     return gamma_hat

# def compute_global_effective_spectral_index(energies, E_lower=energy_range[0]):
#     """
#     Compute the effective spectral index using all the photon energies.
#     This value can be used as a fallback when a particular energy bin is empty.
#     """
#     return effective_spectral_index(energies, E_lower)

# def compute_binned_effective_spectral_indices(energies, num_bins, energy_range=energy_range):
#     """
#     Divide the energy range into logarithmic bins and compute the effective spectral index in each bin.
#     For bins that have no photons, the global effective spectral index (computed from all energies)
#     is used as the fallback.
    
#     Returns a 1D array containing the spectral index for each bin.
#     """
#     # Create logarithmically spaced bins.
#     bins = np.geomspace(energy_range[0], energy_range[1], num_bins + 1)
    
#     # Compute the global effective spectral index from all the energies.
#     global_gamma = compute_global_effective_spectral_index(energies, E_lower=bins[0])
    
#     gamma_bins = []
#     for i in range(len(bins) - 1):
#         # Select energies within current bin: lower inclusive, upper exclusive.
#         mask = (energies >= bins[i]) & (energies < bins[i+1])
#         energies_bin = energies[mask]
        
#         # Compute the effective spectral index for this bin.
#         gamma_bin = effective_spectral_index(energies_bin, E_lower=bins[i])

#         gamma_bins.append(gamma_bin)
    
#     return np.array(gamma_bins), global_gamma

# def compute_summary_statistics(energies, energy_dependent_hist, N_Ebins):
#     """
#     Given an array of photon energies (all between 2 and 100 GeV), compute a set of summary statistics:
#       1. Mean energy.
#       2. Variance of energy.
#       3. Quantiles: 10%, 25%, 50%, 75%, and 90%.
#       NOT DOING: 4. The fraction of photons with energy above E_cross (where E_cross is the crossing point
#          for the two normalized PDFs of the Gamma = 2.2 and Gamma = 2.25 spectra).
#       5. Effective spectral index estimated from the data.
      
#     Returns the statistics in a dictionary and also a flattened torch tensor.
#     """
#     # 1. Mean and variance
#     mean_E, var_E = compute_moments(energies)
    
#     # 2. Quantiles
#     quant_dict = compute_quantiles(energies)  # This returns a dict like {10: val, 25: val, ...}
    
#     # 4. Effective spectral index estimation
#     # gamma_eff = effective_spectral_index(energies, E_lower=energy_range[0])

#     energy_only_hist = compute_energy_only_histogram(energies, num_bins=N_Ebins, energy_range=energy_range)

#     binned_gamma, global_gamma = compute_binned_effective_spectral_indices(energies, num_bins=N_Ebins, energy_range=energy_range)
    
    
#     # If you want to pass the summary statistic to sbi, it is best to use a fixed-size vector (e.g., a torch tensor).
#     # For example, arrange the stats in a consistent order:
#     scalars  = np.array([
#         energies.size, # total number of photons
#         mean_E,
#         var_E,
#         quant_dict[10],
#         quant_dict[25],
#         quant_dict[50],
#         quant_dict[75],
#         quant_dict[90],
#         global_gamma
#     ], dtype=np.float32)
    
#     flat_energy_dependent_hist = np.asarray(energy_dependent_hist, dtype=np.float32).flatten()
#     summary_vector = np.concatenate([flat_energy_dependent_hist, energy_only_hist, binned_gamma, scalars]) # energies.size is the total number of photons

#     # Convert to a torch tensor if needed by sbi.
#     summary_tensor = torch.tensor(summary_vector)
    
#     return summary_tensor

In [12]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-2)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 

    # N_side = 2**6
    # #parameters for the summary statistic
    
    # center_mask = 10 #deg 
    # lat_mask = 5 #deg 
    # N_Ebins = 20
    # Ebinspace = 'log'#'linear'
    # N_countbins = 10
    # countbinspace = 'custom'#'linear'
    # if N_side == 2**6:
    #     mincount, maxcount = 0, 150
    # elif N_side == 2**3:
    #     mincount, maxcount = 0, 5500
    # N_pix = 12*N_side**2
    # pix_i = np.linspace(0, N_pix-1, N_pix, dtype = 'int')
    # roi_pix_i = np.where(np.logical_and(hp.rotator.angdist(np.array([np.pi/2, 0]), hp.pix2ang(N_side, pix_i)) >= center_mask*u.deg.to('rad'), np.abs(np.pi/2 - hp.pix2ang(N_side, pix_i)[0]) >= lat_mask*u.deg.to('rad')))[0]

    # roi_map = my_AEGIS.get_roi_map_summary(photon_info = photon_info, N_side = N_side, N_Ebins = N_Ebins, Ebinspace = Ebinspace, roi_pix_i = roi_pix_i)
    # energy_dependent_hist = my_AEGIS.get_counts_histogram_from_roi_map(roi_map, mincount = mincount, maxcount = maxcount, N_countbins = N_countbins, countbinspace = countbinspace)

    # photon_pixels = hp.ang2pix(N_side, photon_info['angles'][:, 0], photon_info['angles'][:, 1])
    # roi_mask = np.isin(photon_pixels, roi_pix_i)
    # energies_in_roi = photon_info['energies'][roi_mask]


    # stats_tensor = compute_summary_statistics(energies_in_roi, energy_dependent_hist, N_Ebins = N_Ebins)
    
    return photon_info #stats_tensor

In [54]:
def manual_simulate_for_sbi(proposal, num_simulations=1000, num_workers=32):
    """
    Simulates the model in parallel using joblib.
    Each simulation call samples a parameter from the proposal and passes the index to the simulator.
    """
    def run_simulation(i):
        if i % 10 == 0:
            print(f"i= {i}")
        # Sample a parameter from the proposal (sbi.utils.BoxUniform has a .sample() method)
        theta_i = proposal.sample()
        photon_info = simulator(theta_i)
        return theta_i, photon_info

    # Run simulations in parallel using joblib.
    results = Parallel(n_jobs=num_workers, timeout=None)(delayed(run_simulation)(i) for i in range(num_simulations))
    theta_list, photon_info_list = zip(*results)

    theta_tensor = torch.stack(theta_list, dim=0).to(torch.float32)
    
    
    return theta_tensor, photon_info_list

In [55]:
# Define the prior using sbi.utils.BoxUniform
parameter_range = torch.tensor([[-5.0, -7.5 ,-6.5, -6.5],
                                [-1.0, -3.5, -2.5, -2.5]])

prior = utils.BoxUniform(low=parameter_range[0], high=parameter_range[1])

theta, photon_info_list = manual_simulate_for_sbi(prior,
                                   num_simulations=num_simulations,
                                   num_workers=num_workers)

i= 20
i= 90
i= 50
i= 160
i= 110
i= 40
i= 30
i= 130
i= 10
i= 100
i= 150
i= 80
i= 70
i= 60
i= 190
i= 180
i= 120
i= 0
i= 170
i= 140
i= 200
i= 210
i= 220
i= 230
i= 240
i= 250
i= 260
i= 270
i= 280
i= 290
i= 300
i= 310
i= 320
i= 330
i= 340
i= 350
i= 360
i= 370
i= 380


/usr/local/phys/Linux/anaconda3.9/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


i= 390
i= 400
i= 410
i= 420
i= 430
i= 440
i= 450
i= 460
i= 470
i= 480
i= 490
i= 500
i= 510
i= 520
i= 530
i= 540
i= 550
i= 560
i= 570
i= 580
i= 590
i= 600
i= 610
i= 620
i= 630
i= 640
i= 650
i= 660
i= 670
i= 680
i= 690
i= 700
i= 710
i= 720
i= 730
i= 740
i= 750
i= 760
i= 770
i= 780
i= 790
i= 800
i= 810
i= 820
i= 830
i= 840
i= 850
i= 860
i= 870
i= 880
i= 890
i= 900
i= 910
i= 920
i= 930
i= 940
i= 950
i= 960
i= 970
i= 980
i= 990


In [56]:
# 'photon_info_list' is a list of dictionaries

# Save to file
with open('/home/users/ids29/DGRB/photon_info_list.pkl', 'wb') as f:
    pickle.dump(photon_info_list, f)

# Save to file
torch.save(theta, '/home/users/ids29/DGRB/thetas.pt')
torch.save(parameter_range, '/home/users/ids29/DGRB/parameter_range.pt')

In [57]:
input_params = torch.tensor([-2.08, -4.74, -3.25, -3.749])
obs_photon_info = simulator(input_params)

In [58]:
torch.save(input_params, r'/home/users/ids29/DGRB/input_params.pt')

with open(r'/home/users/ids29/DGRB/obs_photon_info.pkl', 'wb') as f:
    pickle.dump(obs_photon_info, f)

Test another observation data where the SFG normalization parameters are two orders of magnitude lesser than above.

In [17]:
input_params_test = torch.tensor([-1.5, -4.0, -3.0, -3.749])
obs_photon_info_test = simulator(input_params)

In [19]:
torch.save(input_params_test, r'/home/users/ids29/DGRB/input_params_test.pt')

with open(r'/home/users/ids29/DGRB/obs_photon_info_test.pkl', 'wb') as f:
    pickle.dump(obs_photon_info_test, f)